In [2]:
%%writefile requirements.txt
transformers==4.44.0
matplotlib==3.9.2
accelerate>=0.20.1
datasets==2.21.0
nlpaug==1.1.10
tensorflow==2.12.0
torch
torchvision
pyngrok
uvicorn
fastapi

Overwriting requirements.txt


In [19]:
!pip install -qr requirements.txt

In [1]:
import uvicorn
from fastapi import FastAPI
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from fastapi import FastAPI, Request, File, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from PIL import Image
from fastapi.responses import JSONResponse
import io

from fastapi import Form
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import transformers
import tensorflow as tf
from nltk.corpus import stopwords
from transformers import DistilBertTokenizer, DistilBertModel, AutoTokenizer, AutoModel, BlipProcessor, BlipForConditionalGeneration, AutoModelForCausalLM, pipeline
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, regularizers
from datasets import Dataset, DatasetDict, load_dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.tokenize import word_tokenize
from transformers import AutoProcessor, AutoModelForVision2Seq

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mourad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mourad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Mourad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mourad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
dataset = pd.read_csv("cellula toxic data.csv")

In [3]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# 1. Cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove non-letters
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# 2. Lemmatization function
def lemmatize_text(text):
    tokens = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(token) for token in tokens])

# 3. Full pipeline
def text_pipeline(text):
    cleaned = clean_text(text)
    lemmatized = lemmatize_text(cleaned)
    return lemmatized


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_ckpt = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
bert_model = DistilBertModel.from_pretrained(model_ckpt, num_labels=9).to(device)

#tokenizer.save_pretrained("./bert_model")
#bert_model.save_pretrained("./bert_model")


ann_model = load_model("model_ann.keras")

c:\Users\Mourad\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
ckpt = "blip-image-captioning-large"

processor = AutoProcessor.from_pretrained("large_blip_model")
blip_model = AutoModelForVision2Seq.from_pretrained("large_blip_model")

#processor.save_pretrained("large_blip_model")
#blip_model.save_pretrained("large_blip_model")


In [7]:


llama_tokenizer = AutoTokenizer.from_pretrained("llama_model", use_auth_token=token)
llama_model = AutoModelForCausalLM.from_pretrained("llama_model", use_auth_token=token, device_map="auto", torch_dtype=torch.float16)

c:\Users\Mourad\anaconda3\lib\site-packages\transformers\models\auto\tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\Mourad\anaconda3\lib\site-packages\transformers\models\auto\auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [8]:
def SoftClassifier(ann_model, bert_model, tokenizer, text):
    text = text_pipeline(text)

    tokenized_sentence = tokenizer(text, return_special_tokens_mask= True, return_tensors='pt', padding=True, truncation=True).to(device)
    input_ids = tokenized_sentence["input_ids"].to(device)
    attention_mask = tokenized_sentence["attention_mask"].to(device)
    special_tokens_mask = tokenized_sentence["special_tokens_mask"].to(device)

    with torch.no_grad():
        outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state


    pooled_embeddings = []
    for i in range(hidden_states.size(0)):
        non_special = ~special_tokens_mask[i].bool()
        clean_embed = hidden_states[i][non_special]  # [num_tokens, 768]
        mean_pooled = clean_embed.mean(dim=0).cpu().numpy()  # [768]
        pooled_embeddings.append(mean_pooled)


    lab2str = {
    0: 'Safe',
    1: 'unsafe',
    2: 'Suicide & Self-Harm',
    3: 'Unknown S-Type',
    4: 'Elections',
    5: 'Violent Crimes',
    6: 'Sex-Related Crimes',
    7: 'Non-Violent Crimes',
    8: 'Child Sexual Exploitation'
     }

    test = pooled_embeddings[0].reshape(1, 768)
    y_pred= ann_model.predict(test)
    y_pred = np.argmax(y_pred, axis=1)

    return lab2str[y_pred[0]]

In [9]:
def image_captioning(blip_model, processor, image):

    # Process the image
    inputs = processor(images=image, return_tensors="pt")
    outputs = blip_model.generate(**inputs)

    caption = processor.decode(outputs[0], skip_special_tokens=True)

    return caption

In [10]:
def classify_text(text):
    pipe = pipeline("text-generation", model=llama_model, tokenizer=llama_tokenizer)

    def get_prompt(input_text):
        return f"""<s>[INST] <<SYS>>
                    You are a text safety classifier. You will classify the following user input as either SAFE or UNSAFE.
                    <</SYS>>

                    Text: {input_text}
                    Classification: [/INST]"""


    response = pipe(get_prompt(text), max_new_tokens=10, return_full_text=False)[0]['generated_text']

    if 'unsafe' in response:
        hard = "unsafe"
        soft = SoftClassifier(ann_model, bert_model, tokenizer, text)

    elif 'safe' in response:
        hard = "safe"
        soft = SoftClassifier(ann_model, bert_model, tokenizer, text)

    else:
        hard = 'unknown'
        soft = 'unknown'


    return hard, soft




In [11]:
from pyngrok import ngrok
import nest_asyncio



app = FastAPI()
templates = Jinja2Templates(directory="templates")
app.mount("/static", StaticFiles(directory="static"), name="static")

@app.get('/', response_class=HTMLResponse)
@app.get('/home', response_class=HTMLResponse)

async def index(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})



@app.get("/predict", response_class=HTMLResponse)
async def get_predict(request: Request):
    return templates.TemplateResponse("predict.html", {"request": request})


@app.post("/predict")
async def post_predict(file: UploadFile = File(...)):
    image_bytes = await file.read()
    image = Image.open(io.BytesIO(image_bytes))
    caption = image_captioning(blip_model, processor, image)
    hard, soft = classify_text(caption)
    return {"caption": caption, "hard": hard, "soft": soft}





@app.post("/classify-text")
async def classify_user_text(user_input: str = Form(...)):
    hard, soft = classify_text(user_input)
    return {"input": user_input, "hard": hard, "soft": soft}




@app.get("/about", response_class=HTMLResponse)
async def about(request: Request):
    return templates.TemplateResponse("about.html", {"request": request})

# Apply asyncio patch
nest_asyncio.apply()

# Open a tunnel on port 8000
public_url = ngrok.connect(8080)
print("Public URL:", public_url)

# Run the app
uvicorn.run(app, port=8080)

Public URL: NgrokTunnel: "https://4e281dd270eb.ngrok-free.app" -> "http://localhost:8080"


INFO:     Started server process [11724]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:54698 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:54698 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54698 - "GET /predict HTTP/1.1" 200 OK


c:\Users\Mourad\anaconda3\lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


1/1 [==============================] - 1s 510ms/step
INFO:     127.0.0.1:54699 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 423ms/step
INFO:     127.0.0.1:54704 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [11724]
